# Ireland Age and Sex Analysis
## Author: Joanna Mnich


In [ ]:
import pandas as pd

In [ ]:
# Upload data
url = "https://ws.cso.ie/public/api.restful/PxStat.Data.Cube_API.ReadDataset/FY006A/CSV/1.0/en"
df = pd.read_csv(url)

# Display first rows to understand structure
df.head()

In [ ]:
df = df[df["Sex"] != "Female"]
df = df[df["Sex"] != "Male"]
df.tail(3)

In [ ]:
headers = df.columns.tolist()
headers

In [ ]:
drop_col_list = ['STATISTIC', 'Statistic Label','TLIST(A1)','CensusYear','C02199V02655','Sex','C02076V03371','C03789V04537','UNIT']
df.drop(columns=drop_col_list, inplace=True)
df = df[df["Single Year of Age"] != "All ages"]
df['Single Year of Age'] = df['Single Year of Age'].str.replace('Under 1 year', '0')
df['Single Year of Age'] = df['Single Year of Age'].str.replace('\D', '', regex=True)

df['Single Year of Age']=df['Single Year of Age'].astype('int64')
df['VALUE']=df['VALUE'].astype('int64')
#print (df.head(3))
df.info()


In [ ]:
df_anal = pd.pivot_table(df, 'VALUE',"Single Year of Age","Administrative Counties")
print (df_anal.head(3))
# write out the entire file to local machine
df_anal.to_csv("population_for_analysis.csv")

In [ ]:
headers = list(df_anal.columns)
district = headers[0]
district

In [ ]:
number_people = df_anal[district].sum()
number_people


In [ ]:
df_anal

In [ ]:
cumages = df_anal[district].mul(df_anal.index, axis=0).sum()
cumages

In [ ]:
weighted_mean = cumages/number_people
weighted_mean

#### Or you can use numpy

In [ ]:
import numpy as np
w_mean = np.average(df_anal.index, weights=df_anal[district])
w_mean

### weighted median

In [ ]:
cumsum = df_anal[district].cumsum()
cumsum

In [ ]:
# sum of all the number of people
cutoff = df_anal[district].sum()/2
cutoff

In [ ]:
df_anal[district][cumsum>=cutoff].index[0]

## Weighted standard deviation

The is the same formula as the normal standard deviation, just with weights applied to the differenced to the weight mean

In [ ]:
# incorrect way
df_anal[district].std()

In [ ]:
# incorrect
df_anal[district].mean()

We need the weighted std

In [ ]:
import numpy as np
w_mean = np.average(df_anal.index, weights=df_anal[district])
w_mean

In [ ]:
w_variance = np.average((df_anal.index - w_mean)**2, weights= df_anal[district])
w_variance

In [ ]:
w_std = np.sqrt(w_variance)
w_std